# Fin Whale call denoising attemp       

In [1]:
# Imports
import sys
sys.path.append('../src/denoiser/')
import os
import gc
import glob
import h5py
import torch
import numpy as np
from tqdm import tqdm
import das4whales as d4w
import matplotlib.pyplot as plt
import scipy.signal as sp
from das_util import *
from denodas_model import unet
import torch.nn as nn


In [14]:


""" Initialize the U-net model """
devc = try_gpu(i=0)
print(devc)

model_1 = unet(1, 16, 1024, factors=(5, 3, 2, 2), use_att=False)
model_1 = nn.DataParallel(model_1, device_ids=[0])
model_1.to(devc)

""" Load the pretrained weights """
model_1.load_state_dict(torch.load(os.path.join(os.getcwd(), 'models/mlmodel/denoiser_weights_LR08_MASK05_raw2raw.pt')))  # raw2raw
# model_1.load_state_dict(torch.load('../models/mlmodel/denoiser_weights_LR08_MASK05_raw2fk.pt'))  # raw2fk
model_1.eval() 

cuda:0


/tmp/ipykernel_522811/536869960.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_1.load_state_dict(torch.load(os.path.join(os.getcwd(), 'models/mlmodel/denoiser_we

FileNotFoundError: [Errno 2] No such file or directory: '/home/qgoestch/Documents/PostDoc/Scripts/Shi_etal_2023_denoiseDAS/examples/models/mlmodel/denoiser_weights_LR08_MASK05_raw2raw.pt'

In [7]:
# Import the data 
filepath = 'sliced_data/North-C1-LR-P1kHz-GL50m-Sp2m-FS200Hz_2021-11-04T020002Z.h5_sliced32.hdf5'

with h5py.File(filepath, 'r') as f:
    rawdata = f['bandpassed_data'][:]

In [ ]:
inp_data = rawdata[:,:,:]
one_denoised = np.zeros_like(inp_data)
mul_denoised = np.zeros_like(inp_data)

for eid in tqdm(np.arange(len(inp_data)), desc='Denoising'):
    one_denoised[eid,:,:], mul_denoised[eid,:,:] = Denoise_largeDAS(inp_data[eid], 
                                                                    model_1, 
                                                                    devc, 
                                                                    repeat=4, 
                                                                    norm_batch=False)


In [ ]:
# Print the results
for i in range(10,20):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(inp_data[i], aspect='auto', cmap='RdBu', origin='lower', vmin=-1, vmax=1)
    ax[0].set_title('Original')
    ax[1].imshow(one_denoised[i], aspect='auto', cmap='RdBu', origin='lower', vmin=-1, vmax=1)
    ax[1].set_title('One denoised')
    ax[2].imshow(mul_denoised[i], aspect='auto', cmap='RdBu', origin='lower', vmin=-1, vmax=1)
    ax[2].set_title('Multiple denoised')
    plt.show()
    plt.close()
    gc.collect()